In [1]:
import os

import networkx as nx
import nltk
from pymongo import MongoClient

In [2]:
client = MongoClient()
# client.drop_database('recipes')
db = client.recipes

In [7]:
def add_node(g, n):
    if not n in g:
        g.add_node(n, count=0)

In [6]:
def add_edge(g, n1, n2):
    if n1 != n2 and not nx.has_path(g, n1, n2):
        g.add_edge(n1, n2)

In [3]:
ingredients_graph = nx.read_gexf('data/elbulli_ingredients_lexicon_1.gexf')

In [4]:
techniques_graph = nx.read_gexf('data/elbulli_techniques_lexicon_1.gexf')

# Representatives

In [9]:
for n in ingredients_graph.nodes():
    if ingredients_graph.node[n]['count'] == 0:
        neighbors = ingredients_graph.neighbors(n)
        ingredients_graph.remove_node(n)
        if len(neighbors) > 1:
            add_edge(ingredients_graph, neighbors[0], neighbors[1])

In [10]:
len(ingredients_graph)

1609

In [11]:
ingredients_graph.number_of_edges()

221

In [12]:
nx.number_connected_components(ingredients_graph)

1388

In [13]:
for n in techniques_graph.nodes():
    if techniques_graph.node[n]['count'] == 0:
        neighbors = techniques_graph.neighbors(n)
        techniques_graph.remove_node(n)
        if len(neighbors) > 1:
            add_edge(techniques_graph, neighbors[0], neighbors[1])

In [14]:
len(techniques_graph)

181

In [15]:
techniques_graph.number_of_edges()

94

In [16]:
nx.number_connected_components(techniques_graph)

87

In [9]:
repr_ingredients_dict = {}

for syns in nx.connected_components(ingredients_graph):
    max_ingr = ''
    max_count = 0
    for ingr in syns:
        dat = ingredients_graph.node[ingr]
        if dat['count'] > max_count:
            max_ingr = ingr
            max_count = dat['count']
    if max_ingr:
        for ingr in syns:
            dat = ingredients_graph.node[ingr]
            if dat['count'] > 0:
                repr_ingredients_dict[ingr] = max_ingr

In [10]:
repr_ingredients_dict['aceite de olivas']

'aceite de oliva'

In [11]:
len(repr_ingredients_dict)

1609

In [12]:
repr_techniques_dict = {}

for syns in nx.connected_components(techniques_graph):
    max_tech = ''
    max_count = 0
    for tech in syns:
        dat = techniques_graph.node[tech]
        if dat['count'] > max_count:
            max_tech = tech
            max_count = dat['count']
    if max_tech:
        for tech in syns:
            dat = techniques_graph.node[tech]
            if dat['count'] > 0:
                repr_techniques_dict[tech] = max_tech

In [13]:
len(repr_techniques_dict)

181

In [14]:
repr_techniques_dict['al horno']

'horno'

In [18]:
len(set(repr_ingredients_dict.values()))

1370

In [19]:
len(set(repr_techniques_dict.values()))

84

In [22]:
db.drop_collection('elbulli')

count = 0
rows = []

for row in db.elbulli_aux.find():
    repr_ingredients = set()
    for ingr in row['ingredients']:
        r = repr_ingredients_dict[ingr]
        repr_ingredients.add(r)
    row['ingredients'] = sorted(repr_ingredients)
    repr_techniques = set()
    for tech in row['techniques']:
        r = repr_techniques_dict[tech]
        repr_techniques.add(r)
    row['techniques'] = sorted(repr_techniques)
    rows.append(row)
    
    count += 1
    if count % 100 == 0:
        db.elbulli.insert_many(rows)
        rows = []
        print(count, 'rows inserted')
db.elbulli.insert_many(rows)
rows = []
print(count, 'rows inserted')

100 rows inserted
200 rows inserted
300 rows inserted
400 rows inserted
500 rows inserted
600 rows inserted
700 rows inserted
800 rows inserted
900 rows inserted
1000 rows inserted
1100 rows inserted
1200 rows inserted
1214 rows inserted
